In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erfc

# Number of bits
N = int(1e6)

# Eb/No range in dB
Eb_No_dB = np.arange(-3, 11)

# Transmit random BPSK symbols: +1 or -1
s = 2 * (np.random.rand(N) > 0.5) - 1

# AWGN noise samples (complex Gaussian, zero-mean, unit-variance per dimension)
n = (np.random.randn(N) + 1j * np.random.randn(N)) / np.sqrt(2)

# Initialize error counters
nErr_AWGN = np.zeros(len(Eb_No_dB))
nErr_Rayleigh = np.zeros(len(Eb_No_dB))

# Loop over each Eb/No value
for i in range(len(Eb_No_dB)):
    noise_scale = 10 ** (-Eb_No_dB[i] / 20)  # Noise scaling for approximate SNR

    # --- AWGN channel ---
    y_AWGN = s + noise_scale * n
    iphat_AWGN = (np.real(y_AWGN) > 0).astype(int)
    nErr_AWGN[i] = np.sum((s > 0) != iphat_AWGN)

    # --- Rayleigh fading channel ---
    h = (np.random.randn(N) + 1j * np.random.randn(N)) / np.sqrt(2)  # Rayleigh fading coefficients
    y_Rayleigh = h * s + noise_scale * n
    iphat_Rayleigh = (np.real(y_Rayleigh / h) > 0).astype(int)
    nErr_Rayleigh[i] = np.sum((s > 0) != iphat_Rayleigh)

# Simulated BERs
simBer_AWGN = nErr_AWGN / N
simBer_Rayleigh = nErr_Rayleigh / N

# Theoretical BERs
Eb_No_lin = 10 ** (Eb_No_dB / 10)
therBer_AWGN = 0.5 * erfc(np.sqrt(Eb_No_lin))
therBer_Rayleigh = 0.5 * (1 - np.sqrt(Eb_No_lin / (1 + Eb_No_lin)))

# --- Plot: AWGN Channel Only ---
plt.figure()
plt.semilogy(Eb_No_dB, therBer_AWGN, 'b.-', label='Theory (AWGN)', linewidth=1.5)
plt.semilogy(Eb_No_dB, simBer_AWGN, 'mx-', label='Simulation (AWGN)', linewidth=1.5)
plt.grid(True, which='both')
plt.axis([-3, 10, 1e-5, 0.5])
plt.xlabel('Eb/No (dB)')
plt.ylabel('Bit Error Rate (BER)')
plt.legend()
plt.title('BPSK BER in AWGN Channel')
plt.show()

# --- Plot: AWGN and Rayleigh ---
plt.figure()
plt.semilogy(Eb_No_dB, therBer_AWGN, 'b.-', label='Theory (AWGN)', linewidth=1.5)
plt.semilogy(Eb_No_dB, simBer_AWGN, 'mx-', label='Simulation (AWGN)', linewidth=1.5)
plt.semilogy(Eb_No_dB, therBer_Rayleigh, 'r.-', label='Theory (Rayleigh)', linewidth=1.5)
plt.semilogy(Eb_No_dB, simBer_Rayleigh, 'go-', label='Simulation (Rayleigh)', linewidth=1.5)
plt.grid(True, which='both')
plt.axis([-3, 10, 1e-5, 0.5])
plt.xlabel('Eb/No (dB)')
plt.ylabel('Bit Error Rate (BER)')
plt.legend()
plt.title('BPSK BER in AWGN and Rayleigh Channel')
plt.show()